In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = 'https://www.techpowerup.com/gpu-specs/'
source = requests.get(url)
print(url)
#print(source.status_code)
#print(source.headers)
src = source.content
#print(src)
soup = BeautifulSoup(src,'lxml')

links = []
for td_tag in soup.find_all('td'):
    a_tag = td_tag.find('a')
    if a_tag != None:
        links.append(a_tag.attrs['href'])
    #print(type(a_tag)
   #a_tag = td_tag.find('a')
    #urls.append(a_tag.attrs['href'])
link_format = links[5][11:]
print(link_format)
next_x = url+link_format

print(next_x)
next_page = requests.get(next_x)
print(next_page.status_code)
#print(next_page)
#next_src = next_page.content
soup2 = BeautifulSoup(next_page.content,'lxml')
#print(soup2.prettify())
print(links)

#MissingSchema: Invalid URL 'next_x': No schema supplied. Perhaps you meant http://next_x?
#https://www.techpowerup.com/gpu-specs/geforce-rtx-3060.c3682
'''
for i in soup.find_all('a', href = True):
    
  # check all link which is contain
  # "www.geeksforgeeks.org" string 
  if("www.geeksforgeeks.org" in i['href']):
      
    # call get method to request next url
    nextpage = requests.get(i['href'])
      
    # create soup for next url
    nextsoup = BeautifulSoup(nextpage.content, 'html.parser')
      
    # we can scrap any thing of the
    # next page here we are scraping title of 
    # nexturl page string
    print("next url title : ",nextsoup.find('title').string)
'''

https://www.techpowerup.com/gpu-specs/
radeon-rx-7900-xt.c3912
https://www.techpowerup.com/gpu-specs/radeon-rx-7900-xt.c3912
200
['/gpu-specs/radeon-rx-6400.c3813', '/gpu-specs/radeon-rx-6400.c3813', '/gpu-specs/radeon-rx-6400.c3813', '/gpu-specs/geforce-rtx-2050-mobile.c3859', '/gpu-specs/radeon-rx-6650-xt.c3898', '/gpu-specs/radeon-rx-7900-xt.c3912', '/gpu-specs/radeon-hd-5870.c253', '/gpu-specs/radeon-rx-6650-xt.c3898', '/gpu-specs/radeon-rx-6950-xt.c3875', '/gpu-specs/radeon-rx-7700-xt.c3911', '/gpu-specs/radeon-rx-7800-xt.c3839', '/gpu-specs/radeon-rx-7900-xt.c3912', '/gpu-specs/radeon-rx-7700-xt.c3911', '/gpu-specs/geforce-rtx-3050-ti-mobile.c3812', '/gpu-specs/radeon-rx-6750-xt.c3879', '/gpu-specs/arc-7-a780.c3910', '/gpu-specs/arc-7-a780.c3910', '/gpu-specs/arc-3-a350m.c3905', '/gpu-specs/arc-3-a370m.c3906', '/gpu-specs/arc-5-a550m.c3907', '/gpu-specs/arc-7-a730m.c3908', '/gpu-specs/arc-7-a770m.c3909', '/gpu-specs/geforce-rtx-3090-ti.c3829', '/gpu-specs/arc-7-a770m.c3909', '/gp

'\nfor i in soup.find_all(\'a\', href = True):\n    \n  # check all link which is contain\n  # "www.geeksforgeeks.org" string \n  if("www.geeksforgeeks.org" in i[\'href\']):\n      \n    # call get method to request next url\n    nextpage = requests.get(i[\'href\'])\n      \n    # create soup for next url\n    nextsoup = BeautifulSoup(nextpage.content, \'html.parser\')\n      \n    # we can scrap any thing of the\n    # next page here we are scraping title of \n    # nexturl page string\n    print("next url title : ",nextsoup.find(\'title\').string)\n'

In [26]:
title = soup2.find('h1', class_="gpudb-name").text
#specs = soup2.find('dd', class_="gpudb-specs-large__value")
specs1 = soup2.find('div').find_all('dl')
#specs2 = soup2.find('dl', class_="clearfix").find_all('div')
#specs1 = soup2.find('dl', class_="gpudb-specs-large").find_all('div')
#print(title)
list_info = []
list_info_heading = []
for spec1 in specs1:
    basic_info = spec1.find('dd').text
    basic_heading = spec1.find('dt').text
    list_info.append(basic_info)
    list_info_heading.append(basic_heading)
    
print(list_info_heading)
print(list_info)
    
print(title)

    
    #test = spec.find('dd', class_='gpudb-specs-large__value').text
    #print(list_value) ,class_='gpudb-specs-large__value'
    # ,class_='gpudb-specs-large__title'

#print(specs1)
    
#movies = soup.find('tbody', class_="lister-list").find_all('tr')

['Graphics Processor', 'GPU Name', 'GPU Variant', 'Architecture', 'Foundry', 'Process Size', 'Transistors', 'Die Size', 'Release Date', 'Generation', 'Predecessor', 'Production', 'Bus Interface', 'Base Clock', 'Game Clock', 'Boost Clock', 'Memory Clock', 'Memory Size', 'Memory Type', 'Memory Bus', 'Bandwidth', 'Shading Units', 'TMUs', 'ROPs', 'Compute Units', 'RT Cores', 'L0 Cache', 'L1 Cache', 'L2 Cache', 'L3 Cache', 'Pixel Rate', 'Texture Rate', 'FP16 (half) performance', 'FP32 (float) performance', 'FP64 (double) performance', 'Slot Width', 'Length', 'Width', 'Height', 'TDP', 'Suggested PSU', 'Outputs', 'Power Connectors', 'Part Number', 'DirectX', 'OpenGL', 'OpenCL', 'Vulkan', 'Shader Model']
['Navi 31', '\nNavi 31\n', '\n\t\t\t\t\tNavi 31 XT\n\t\t\t\t\t\t\t\t\t', 'RDNA 3.0', 'TSMC', '5 nm', 'unknown', 'unknown', '2022', '\nNavi III\n(RX 7000)\t\t\t\t', '\nNavi II\n', 'Unreleased', 'PCIe 5.0 x16', '1800 MHz', '1900 MHz', '2200 MHz', '\n\t\t\t\t\t2250 MHz\n\t\t\t\t\t\t\t\t\t\t\n\t\t

In [8]:
cores = spec.find('dd').text
    tmu = spec.find('dd').get_text()
    rops = spec.find('dd').get_text()
    vram = spec.find('dd').get_text()
    memtype = spec.find('dd').get_text()
    bus = spec.find('dd').get_text()
    list_value = [gpu_model, cores, tmu, rops, vram, memtype, bus]

for movie in movies:
        
        name = movie.find('td', class_='titleColumn').a.text
        rank = movie.find('td', class_='titleColumn').get_text(strip=True).split('.')[0]
        year = movie.find('td', class_='titleColumn').span.text.strip('()')
        rating = movie.find('td', class_='ratingColumn imdbRating').strong.text

IndentationError: unexpected indent (Temp/ipykernel_664/3385313547.py, line 2)

In [ ]:
print(urls[0].strip())

In [ ]:
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; their names:
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
<b class="boldest">Extremely bold</b>
<blockquote class="boldest">Extremely bold</blockquote>
<b id="1">Test 1</b>
<b another-attribute="1" id="verybold">Test 2</b>
"""


In [ ]:
with open('index.html', 'w') as f:
    f.write(html_doc)

In [ ]:
soup = BeautifulSoup(html_doc, 'lxml')
#print(soup.prettify())

In [ ]:
print(soup.b)

In [ ]:
print(soup.find_all('b'))

In [ ]:
print(soup.b.name)

In [ ]:
tag = soup.b
print(tag)
tag.name = 'blockquote'
print(tag)

In [ ]:
tag = soup.find_all('b')[1]
print(tag)

In [ ]:
print(tag['id'])

In [ ]:
tag = soup.find_all('b')[2]
print(tag)

In [ ]:
print(tag['id'])
print(tag['another-attribute'])

In [ ]:
tag = soup.find_all('b')[2]
print(tag)
print(tag.attrs)

In [ ]:
print(tag)
tag['another-attribute'] = 2
print(tag)

In [ ]:
print(tag)
del tag['id']
del tag['another-attribute']
print(tag)

In [ ]:
tag = soup.find_all('b')[2]
print(tag)
print(tag.string)

In [ ]:
print(tag)
tag.string.replace_with("This is another string")
print(tag)

In [ ]:
source = requests.get('https://www.techpowerup.com/gpu-specs/')
#print(source.status_code)
#print(source.headers)
src = source.content
#print(src)
soup = BeautifulSoup(src,'lxml')

urls = []
for td_tag in soup.find_all('td'):
    a_tag = td_tag.find('a')
    if a_tag != None:
        urls.append(a_tag.attrs['href'])
    #print(type(a_tag)
   #a_tag = td_tag.find('a')
    #urls.append(a_tag.attrs['href'])
    
print(urls)

In [ ]:
source = requests.get('https://www.techpowerup.com/gpu-specs/')

#print(source.status_code)
#print(source.headers)

src = source.content
#print(src)
soup = BeautifulSoup(src,'lxml')

links = soup.find_all('a')
#print(links)
#print("\n")
for link in links:
    #if "About" in link.text:
        #print(link)
    print(link.attrs['href'])
    #print()

In [ ]:
title = gpu.find('td', class_="vendor-NVIDIA")

In [ ]:
print(gpu)

In [ ]:
<td class="vendor-NVIDIA">
		<a href="/gpu-specs/geforce-rtx-3060.c3682">GeForce RTX 3060</a>

			</td>

In [ ]:
try:
    source = requests.get('https://www.imdb.com/chart/top/')
    source.raise_for_status()
    soup = BeautifulSoup(source.text,'html.parser')
    movies = soup.find('tbody', class_="lister-list").find_all('tr')
    print(movies)
    df = pd.DataFrame(columns=['Rank','Name','Year','Rating'])
    temp_dict = {}
    temp_list = []
    
    for movie in movies:
        name = movie.find('td', class_='titleColumn').a.text
        rank = movie.find('td', class_='titleColumn').get_text(strip=True).split('.')[0]
        year = movie.find('td', class_='titleColumn').span.text.strip('()')
        rating = movie.find('td', class_='ratingColumn imdbRating').strong.text
        print(name)
        list_key = ['Rank','Name','Year','Rating']
        list_value = [rank, name, year, rating]
        
        for key, value in zip(list_key, list_value):
            temp_dict[key] = value
            
        df = df.append(temp_dict, ignore_index=True, sort=False)
   
except Exception as e:
    print(e)
    


In [ ]:
df.head()